### Applying normalization, resampling, and reprojection to Match SpaceNet 6 Resolution

In [1]:
import os
import rasterio as ras
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
# Define the directory containing .tif files (converted from jp2 to tiffs in 3_0)
input_dop_folder = "Z:/BuildingDetection/dopJP2_to_tiff_2" 
output_dop_folder = "Z:/BuildingDetection/dop_Reproject"

os.makedirs(output_dop_folder, exist_ok=True)  # Ensure the output folder exists

In [3]:
target_crs = "EPSG:4326"

# Get a sorted list of .tif files in the folder
all_files = [f for f in os.listdir(input_dop_folder) if f.endswith((".tif", ".tiff"))]

In [4]:

# Process each file
for filename in all_files:
    input_path = os.path.join(input_dop_folder, filename)
    output_path = os.path.join(output_dop_folder, filename)

    with ras.open(input_path) as src:
        # Debug input file
        print(f"Processing file: {input_path}")
        print(f"Input CRS: {src.crs}, Bounds: {src.bounds}")

        # Calculate the transform for reprojection
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        
        # Update metadata for the new file
        new_meta = src.meta.copy()
        new_meta.update({
            "crs": target_crs,
            "transform": transform,
            "width": width,
            "height": height
        })

        # Create the output file and reproject
        with ras.open(output_path, "w", **new_meta) as dst:
            for band in range(1, src.count + 1):
                reproject(
                    source=ras.band(src, band),
                    destination=ras.band(dst, band),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest  # Use nearest neighbor for simplicity
                )

    print(f"Reprojected and saved: {output_path}")

print("Reprojection complete!")


Processing file: Z:/BuildingDetection/dopJP2_to_tiff_2\dop10rgbi_32_280_5652_1_nw_2023.tiff
Input CRS: EPSG:25832, Bounds: BoundingBox(left=280000.0, bottom=5652000.0, right=281000.0, top=5653000.0)
Reprojected and saved: Z:/BuildingDetection/dop_Reproject\dop10rgbi_32_280_5652_1_nw_2023.tiff
Processing file: Z:/BuildingDetection/dopJP2_to_tiff_2\dop10rgbi_32_280_5653_1_nw_2023.tiff
Input CRS: EPSG:25832, Bounds: BoundingBox(left=280000.0, bottom=5653000.0, right=281000.0, top=5654000.0)
Reprojected and saved: Z:/BuildingDetection/dop_Reproject\dop10rgbi_32_280_5653_1_nw_2023.tiff
Processing file: Z:/BuildingDetection/dopJP2_to_tiff_2\dop10rgbi_32_280_5654_1_nw_2023.tiff
Input CRS: EPSG:25832, Bounds: BoundingBox(left=280000.0, bottom=5654000.0, right=281000.0, top=5655000.0)
Reprojected and saved: Z:/BuildingDetection/dop_Reproject\dop10rgbi_32_280_5654_1_nw_2023.tiff
Processing file: Z:/BuildingDetection/dopJP2_to_tiff_2\dop10rgbi_32_280_5655_1_nw_2023.tiff
Input CRS: EPSG:25832, Bou

In [5]:
print(f"Transform: {transform}")
print(f"Width: {width}, Height: {height}")


Transform: | 0.00, 0.00, 5.86|
| 0.00,-0.00, 51.06|
| 0.00, 0.00, 1.00|
Width: 12240, Height: 7714
